# GPT calling test

In [ ]:
# OpenAI API 설정 및 라이브러리 import
import openai
from openai import OpenAI
import json
import os
from typing import List, Dict, Any
import dotenv

dotenv.load_dotenv()
# API 키 설정 (환경변수에서 가져오기)
client = OpenAI(
    api_key=os.getenv("OPENAI_API_KEY")  # 환경변수에 API 키를 설정해주세요
)

## Chat

Args:
        - messages: 대화 메시지 리스트 [{"role": "user", "content": "메시지"}]
        - model: 사용할 모델명 (기본값: gpt-4o-mini)
        - **kwargs: OpenAI API 매개변수들

In [ ]:
def chat(
    messages: List[Dict[str, str]], 
    model: str = "gpt-4o-mini",
    **kwargs
) -> str:
    """
    OpenAI GPT-4o-mini API를 사용하여 채팅 완성을 수행합니다.
    
    Args:
        messages: 대화 메시지 리스트 [{"role": "user", "content": "메시지"}]
        model: 사용할 모델명 (기본값: gpt-4o-mini)
        **kwargs: OpenAI API 매개변수들
            - temperature: 창의성 조절 (0.0-2.0, 기본값: 0.7)
            - max_tokens: 최대 토큰 수 (기본값: 1000)
            - top_p: 확률 임계값 (기본값: 0.95)
            - frequency_penalty: 빈도 페널티 (기본값: 0.0)
            - presence_penalty: 존재 페널티 (기본값: 0.0)
            - stream: 스트리밍 여부 (기본값: False)
            - 기타 OpenAI API가 지원하는 모든 매개변수
        
    Returns:
        GPT 응답 텍스트
    """
    # 기본값 설정
    default_params = {
        "temperature": 0.7,
        "max_tokens": 1000,
        "top_p": 0.95,
        "frequency_penalty": 0.0,
        "presence_penalty": 0.0
    }
    
    # 기본값과 사용자 입력 병합
    params = {**default_params, **kwargs}
    
    try:
        response = client.chat.completions.create(
            model=model,
            messages=messages,
            **params
        )
        
        return response.choices[0].message.content
        
    except Exception as e:
        print(f"API 호출 중 오류 발생: {e}")
        return None


In [ ]:
# 간단한 사용 예제
def simple_chat_example():
    """간단한 채팅 예제"""
    
    # 단일 메시지 예제
    messages = [
        {"role": "system", "content": "당신은 도움이 되는 AI 어시스턴트입니다. 한국어로 답변해주세요."},
        {"role": "user", "content": "안녕하세요! 파이썬에 대해 간단히 설명해주세요."}
    ]
    
    response = chat(messages)
    print("=== 단일 메시지 응답 ===")
    print(response)
    print("\n" + "="*50 + "\n")
    
    # # 대화 컨텍스트를 유지한 다중 메시지 예제
    # conversation = [
    #     {"role": "system", "content": "당신은 도움이 되는 AI 어시스턴트입니다. 한국어로 답변해주세요."},
    #     {"role": "user", "content": "머신러닝이 무엇인지 설명해주세요."},
    #     {"role": "assistant", "content": "머신러닝은 컴퓨터가 데이터로부터 패턴을 학습하여 예측이나 결정을 내리는 인공지능의 한 분야입니다."},
    #     {"role": "user", "content": "그렇다면 딥러닝과의 차이점은 무엇인가요?"}
    # ]
    
    # response = chat(conversation)
    # print("=== 대화 컨텍스트 유지 응답 ===")
    # print(response)

# 예제 실행
simple_chat_example()


## Stream

In [ ]:
# 스트리밍 응답 함수
def chat_with_streaming(
    messages: List[Dict[str, str]], 
    model: str = "gpt-4o-mini",
    temperature: float = 0.7,
    max_tokens: int = 1000
):
    """
    스트리밍 방식으로 GPT-4o-mini API 응답을 받습니다.
    실시간으로 응답을 출력합니다.
    """
    try:
        stream = client.chat.completions.create(
            model=model,
            messages=messages,
            temperature=temperature,
            max_tokens=max_tokens,
            stream=True
        )
        
        print("=== 스트리밍 응답 ===")
        full_response = ""
        
        for chunk in stream:
            if chunk.choices[0].delta.content is not None:
                content = chunk.choices[0].delta.content
                print(content, end="", flush=True)
                full_response += content
        
        print("\n" + "="*50)
        return full_response
        
    except Exception as e:
        print(f"스트리밍 API 호출 중 오류 발생: {e}")
        return None

# 스트리밍 예제
def streaming_example():
    messages = [
        {"role": "user", "content": "인공지능의 미래에 대해 200자 정도로 설명해주세요."}
    ]
    
    response = chat_with_streaming(messages)
    
streaming_example()  # 주석을 해제하여 실행


## Function Call

In [ ]:
# 고급 기능: 함수 호출 (Function Calling)
def chat_with_function_calling():
    """
    OpenAI의 Function Calling 기능을 사용한 예제
    """
    
    # 함수 정의
    functions = [
        {
            "name": "get_weather",
            "description": "특정 위치의 날씨 정보를 가져옵니다",
            "parameters": {
                "type": "object",
                "properties": {
                    "location": {
                        "type": "string",
                        "description": "도시 이름 (예: 서울, 부산)"
                    },
                    "unit": {
                        "type": "string",
                        "enum": ["celsius", "fahrenheit"],
                        "description": "온도 단위"
                    }
                },
                "required": ["location"]
            }
        }
    ]
    
    messages = [
        {"role": "user", "content": "서울의 오늘 날씨는 어떤가요?"}
    ]
    
    try:
        response = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=messages,
            functions=functions,
            function_call="auto"
        )
        
        message = response.choices[0].message
        
        if message.function_call:
            print("=== 함수 호출 감지 ===")
            print(f"함수명: {message.function_call.name}")
            print(f"매개변수: {message.function_call.arguments}")
            
            # 실제로는 여기서 함수를 실행하고 결과를 반환해야 합니다
            function_result = "서울의 현재 날씨는 맑음, 기온 22도입니다."
            
            # 함수 결과를 메시지에 추가
            messages.append({
                "role": "function",
                "name": "get_weather",
                "content": function_result
            })
            
            # 최종 응답 생성
            final_response = client.chat.completions.create(
                model="gpt-4o-mini",
                messages=messages
            )
            
            print("=== 최종 응답 ===")
            print(final_response.choices[0].message.content)
        else:
            print("=== 일반 응답 ===")
            print(message.content)
            
    except Exception as e:
        print(f"Function Calling API 호출 중 오류 발생: {e}")

chat_with_function_calling()  # 주석을 해제하여 실행


In [ ]:

# 배치 처리 및 비용 효율적인 사용
def batch_chat_requests(questions: List[str], system_prompt: str = None):
    """
    여러 질문을 배치로 처리하는 함수
    """
    responses = []
    
    for i, question in enumerate(questions):
        print(f"처리 중... ({i+1}/{len(questions)})")
        
        messages = []
        if system_prompt:
            messages.append({"role": "system", "content": system_prompt})
        messages.append({"role": "user", "content": question})
        
        response = chat(
            messages=messages,
            temperature=0.3,  # 일관된 답변을 위해 낮은 temperature
            max_tokens=500
        )
        
        responses.append({
            "question": question,
            "response": response
        })
    
    return responses

# 배치 처리 예제
def batch_example():
    questions = [
        "파이썬의 주요 특징 3가지는?",
        "머신러닝과 딥러닝의 차이점은?",
        "API란 무엇인가요?"
    ]
    
    system_prompt = "당신은 기술 교육 전문가입니다. 간결하고 이해하기 쉽게 설명해주세요."
    
    results = batch_chat_requests(questions, system_prompt)
    
    print("=== 배치 처리 결과 ===")
    for i, result in enumerate(results, 1):
        print(f"\n질문 {i}: {result['question']}")
        print(f"답변: {result['response']}")
        print("-" * 50)

batch_example()  # 주석을 해제하여 실행


In [ ]:
# 에러 처리 및 재시도 로직
import time
import random
from functools import wraps

def retry_on_failure(max_retries: int = 3, delay: float = 1.0):
    """
    API 호출 실패 시 재시도하는 데코레이터
    """
    def decorator(func):
        @wraps(func)
        def wrapper(*args, **kwargs):
            for attempt in range(max_retries):
                try:
                    return func(*args, **kwargs)
                except Exception as e:
                    if attempt == max_retries - 1:
                        print(f"최대 재시도 횟수 ({max_retries})에 도달했습니다. 마지막 오류: {e}")
                        raise e
                    
                    wait_time = delay * (2 ** attempt) + random.uniform(0, 1)
                    print(f"시도 {attempt + 1} 실패: {e}")
                    print(f"{wait_time:.2f}초 후 재시도...")
                    time.sleep(wait_time)
            
        return wrapper
    return decorator

@retry_on_failure(max_retries=3)
def robust_chat_request(messages: List[Dict[str, str]], **kwargs):
    """
    에러 처리와 재시도 로직이 포함된 안정적인 채팅 요청 함수
    """
    try:
        response = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=messages,
            **kwargs
        )
        
        return response.choices[0].message.content
        
    except openai.RateLimitError as e:
        print(f"Rate limit 에러: {e}")
        raise
    except openai.APIConnectionError as e:
        print(f"API 연결 에러: {e}")
        raise
    except openai.InvalidRequestError as e:
        print(f"잘못된 요청: {e}")
        raise
    except Exception as e:
        print(f"기타 오류: {e}")
        raise

# 에러 처리 예제
def error_handling_example():
    messages = [
        {"role": "user", "content": "안전한 API 호출 테스트입니다."}
    ]
    
    try:
        response = robust_chat_request(messages, temperature=0.7, max_tokens=100)
        print("=== 안전한 API 호출 성공 ===")
        print(response)
    except Exception as e:
        print(f"최종 실패: {e}")

# error_handling_example()  # 주석을 해제하여 실행
